# **House Price Prediction**

In oder to build a model that will predict the house price in Ames, we followed a certain outline. We split the process into the following contents:
1. Importing libraries and Loading data
2. Exploratory data analysis
3. different models experimented
4. chosen model

We first needed to import most libraries needed and the data. Additional libraries will be imported as needed in the notebook. 

# 1. Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

import numpy as np        # linear algebra
import pandas as pd       # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns     # data visualisation 

from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso


import datetime     # datetime for calculating the age of things
import scipy.stats as st
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Data sets
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

# 2. Exploratory data analysis


We first skeemed throught the variables and realised that we had two more columns than we thought. The Id column is not needed in our model therefore we can remove it. We will save the Id column first because we will need it for the submission.

In [ ]:
#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

In [ ]:
#We drop the ID columns in both datasets
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

The next step was to look at the sale price, which is the varable we are investigating. Plotting a histogram showed that the data was negatively skewed distribution. Then it was normalised using log.

In [ ]:
#Normalising the data
sns.distplot(train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)
plt.show()

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column 
train["SalePrice"] = np.log1p(train["SalePrice"])

#Check the new distribution 
sns.distplot(train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)
plt.show()

Then we to a look at the relationship between the different variables and sale price. We did this to help us identify and remove outliers. 

In [ ]:
#Outliers
fig, ax = plt.subplots()
ax.scatter(x = train['GrLivArea'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

In [ ]:
sorted_train = train.sort_values(by = ['SalePrice'])
q1,q3 = np.percentile(sorted_train['SalePrice'], [25,75])
iqr = q3-q1
lower_bound = q1 - (1.5*iqr)
upper_bound = q3 + (1.5*iqr)

In [ ]:
lower_bound

In [ ]:
upper_bound

In [ ]:
#Deleting outliers
train = train.drop(train[(train['GrLivArea']>lower_bound) & (train['SalePrice']<upper_bound)].index)

#Check the graphic again
fig, ax = plt.subplots()
ax.scatter(train['GrLivArea'], train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

The data provided is already split into train and test. To clean the data all at once, we first combined the data, cleaned it and then split it again. This helped for both train and test data to be cleaned at once and in the same way.

In [ ]:
#combining tests and train into one set to avoid code repetition
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

In [ ]:
#missing data
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

In [ ]:
#visual representation of the misisng data
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
#missing data
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")
    
#Special fills
#zeros
for col in ('GarageYrBlt','GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0) #we don't fill GarageYrBlt we create GarageAge for it.
            
all_data['GarageAge'] = datetime.datetime.now().year - all_data['GarageYrBlt'] 
all_data.drop(columns=['GarageYrBlt'], inplace = True)
            
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

#median fill
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

#mode fill
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])


#explained by data_description.txt
all_data["Functional"] = all_data["Functional"].fillna("Typ")

#drops
all_data = all_data.drop(['Utilities'], axis=1)


In [ ]:
#Check remaining missing values if any 
all_data[col] = all_data[col].fillna(0)

all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

The next step was to find correlation between variables and then with sale price.

In [ ]:
#Correlation map to see how features are correlated with SalePrice
corrmat = train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True)

If we add first, second and basement surface area, we get total surface area of the house. As a result of this, we can drop the column second floor.

In [ ]:
#New column TotalSF to keep some of the influence the soon to dropped column '2ndFlrSF'
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

For the ordinal and cartegorical values, we replaved them with numerical values to make them a numerical. This was done to each specific variable.

In [ ]:
replace_map_landslope = {'Sev': 3, 'Mod': 2, 'Gtl': 1  }
replace_map_qual =  {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2,'Po': 1 }
replace_map_qual_na = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2,'Po': 1, 'None': 0  }
replace_map_finish = {'GLQ': 6,'ALQ': 5,'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'None':0 }
replace_map_exposure = {'Gd': 4,'Av': 3,'Mn': 2, 'No': 1, 'None': 0 }
replace_map_yes_no = {'Y': 1, 'N': 0 }
replace_map_functional = { 'Typ': 8, 'Min1': 7, 'Min2':6, 'Mod':5, 'Maj1':4, 'Maj2': 3, 'Sev': 2, 'Sal':1 }
replace_map_garage_fin = { 'Fin': 3, 'RFn': 2 , 'Unf': 1, 'None': 0 }
replace_map_drive = { 'Y': 2 , 'P': 1 ,'N': 0 }
replace_map_pool = { 'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'None': 0 }
replace_map_fence = {'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'None': 0 }

In [ ]:
def get_dict_value(key, replace_dict):
    '''Check if key is in dictionary
       if not the function must do nothing
       if the key is in the dictionary, the function must return the corresponding value'''
    if key in replace_dict:
        return replace_dict[key]
    else:
        return key

In [ ]:
#ExterQual
all_data['ExterQual'] = all_data['ExterQual'].apply(get_dict_value, replace_dict = replace_map_qual ) 
all_data['ExterQual'] = all_data['ExterQual'].astype('int')
#ExterCond
all_data['ExterCond'] = all_data['ExterCond'].apply(get_dict_value, replace_dict = replace_map_qual ) 
all_data['ExterCond'] = all_data['ExterCond'].astype('int')
#LandSlope
all_data['LandSlope'] = all_data['LandSlope'].apply(get_dict_value, replace_dict = replace_map_landslope ) 
all_data['LandSlope'] = all_data['LandSlope'].astype('int')
#BsmtQual
all_data['BsmtQual'] = all_data['BsmtQual'].apply(get_dict_value, replace_dict = replace_map_qual_na ) 
all_data['BsmtQual'] = all_data['BsmtQual'].astype('int')
#BsmtCond
all_data['BsmtCond'] = all_data['BsmtCond'].apply(get_dict_value, replace_dict = replace_map_qual_na ) 
all_data['BsmtCond'] = all_data['BsmtCond'].astype('int')
#BsmtFinType1
all_data['BsmtFinType1'] = all_data['BsmtFinType1'].apply(get_dict_value, replace_dict = replace_map_finish ) 
all_data['BsmtFinType1'] = all_data['BsmtFinType1'].astype('int')
#BsmtFinType2
all_data['BsmtFinType2'] = all_data['BsmtFinType2'].apply(get_dict_value, replace_dict = replace_map_finish ) 
all_data['BsmtFinType2'] = all_data['BsmtFinType2'].astype('int')
#HeatingQC
all_data['HeatingQC'] = all_data['HeatingQC'].apply(get_dict_value, replace_dict = replace_map_qual ) 
all_data['HeatingQC'] = all_data['HeatingQC'].astype('int')
#CentralAir
all_data['CentralAir'] = all_data['CentralAir'].apply(get_dict_value, replace_dict = replace_map_yes_no ) 
all_data['CentralAir'] = all_data['CentralAir'].astype('int')
#KitchenQual
all_data['KitchenQual'] = all_data['KitchenQual'].apply(get_dict_value, replace_dict = replace_map_qual_na ) 
all_data['KitchenQual'] = all_data['KitchenQual'].astype('int')
#Functional
all_data['Functional'] = all_data['Functional'].apply(get_dict_value, replace_dict = replace_map_functional ) 
all_data['Functional'] = all_data['Functional'].astype('int')
#FireplaceQu
all_data['FireplaceQu'] = all_data['FireplaceQu'].apply(get_dict_value, replace_dict = replace_map_qual_na ) 
all_data['FireplaceQu'] = all_data['FireplaceQu'].astype('int')
#GarageFinish
all_data['GarageFinish'] = all_data['GarageFinish'].apply(get_dict_value, replace_dict = replace_map_garage_fin) 
all_data['GarageFinish'] = all_data['GarageFinish'].astype('int')
#GarageQual
all_data['GarageQual'] = all_data['GarageQual'].apply(get_dict_value, replace_dict = replace_map_qual_na) 
all_data['GarageQual'] = all_data['GarageQual'].astype('int')
#GarageCond
all_data['GarageCond'] = all_data['GarageCond'].apply(get_dict_value, replace_dict = replace_map_qual_na) 
all_data['GarageCond'] = all_data['GarageCond'].astype('int')

Now we check our data again for skewness. This is because if the dataset is skewed, then the ML model would not a good job of predicting.

In [ ]:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head()

In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)
    
#all_data[skewed_features] = np.log1p(all_data[skewed_features])  #In case Box cox doesn't work
#x = exp(log(lambda * transform + 1) / lambda)                    #To transform the values back

There are some nominal values in our data that we need to get numerical values for them. We use .get_dummies to convert categorical variable into dummy/indicator variables.

In [ ]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

In [ ]:
all_data.head()

We are now ready to build our model, we have cleaned our data enough. To start we split the data back to train and test

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]

Just incase we check for missing data one more time before we start with our model. And for some reason, Garage age has missing data.

In [ ]:

#missing data
test_na = (test.isnull().sum() / len(test)) * 100
test_na = test_na.drop(test_na[test_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame( {'Missing Ratio' :test_na} )
missing_data.head(20)

In [ ]:
test['GarageAge'] = test['GarageAge'].fillna(0)

# 3. Models

In [ ]:
sc=RobustScaler()
x=sc.fit_transform(train)
test=sc.transform(test)

In [ ]:
test

In [ ]:
model = Lasso(alpha =0.001, random_state=1)
model.fit(x,y_train)
pred=model.predict(test)
preds=np.exp(pred)

In [ ]:
pred


In [ ]:
output=pd.DataFrame({'Id':test_ID.values, 'SalePrice':preds})
output.head(10)

In [ ]:
output.to_csv('submission.csv', index=False)